____________________________
##This Python tool is designed to automatically analyze and compare Raman spectra from unknown and known polymer samples. It helps in identifying materials based on spectral similarity, combining peak-based matching and full-spectrum correlation

____________________

### - Automatic peak detection using scipy.signal.find_peaks

### - Peak matching within a user-defined tolerance

### - Full-spectrum (or selected range) comparison using Pearson correlation

### - Weighted similarity scoring based on peak match and spectrum shape

### - Visualization of best matches with overlaid spectra and metric breakdown

### - Excel report generation with conditional formatting of matches

### - Fast processing of multiple files using folder inputs

__________________________


## 1.0- Baseline Removal and Smoothing only (no peak picking)
##### Do this step for both known and unknown signals if they are not corrected 

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from scipy import sparse
from scipy.sparse.linalg import spsolve
from scipy.signal import savgol_filter
import os
from BaselineRemoval import BaselineRemoval
import glob

# Function to estimate baseline using ALS
def baseline_als(y, lam, p, niter=10):
    L = len(y)
    D = sparse.csc_matrix(np.diff(np.eye(L), 2))
    w = np.ones(L)
    for i in range(niter):
        W = sparse.spdiags(w, 0, L, L)
        Z = W + lam * D.dot(D.transpose())
        z = spsolve(Z, w*y)
        w = p * (y > z) + (1-p) * (y < z)
    return z

# Function to load data from different file formats
def load_data(file_path):
    file_ext = os.path.splitext(file_path)[1].lower()
    
    if file_ext == '.csv':
        data = pd.read_csv(file_path)
    elif file_ext == '.xlsx':
        data = pd.read_excel(file_path)
    elif file_ext == '.txt':
        data = pd.read_csv(file_path, sep='\t')  # Assuming tab-separated values
    else:
        raise ValueError(f"Unsupported file format: {file_ext}")
    
    # Check if the data has the expected columns
    if 'Wave' not in data.columns or 'Intensity' not in data.columns:
        # Try to handle files with different column names
        if len(data.columns) >= 2:
            # Rename the first two columns to 'Wave' and 'Intensity'
            data = data.iloc[:, :2]
            data.columns = ['Wave', 'Intensity']
        else:
            raise ValueError(f"File does not have the required columns: {file_path}")
    
    return data

# Function to process a single file and return the processed data
def process_file(file_path, method, poly_degree, iters, conv_thresh, lambda_zhang, porder, repitition, 
                 lam_als, p_als, window_length, poly_order, output_dir, visualize=False):
    
    try:
        # Load the data
        data = load_data(file_path)
        
        # Apply baseline removal
        if method == 'ALS':
            baseline = baseline_als(data['Intensity'].values, lam_als, p_als)
            corrected_intensity = data['Intensity'].values - baseline
        else:
            baseObj = BaselineRemoval(data['Intensity'].values)
            if method == 'ModPoly':
                corrected_intensity = baseObj.ModPoly(poly_degree, iters, conv_thresh)
                baseline = data['Intensity'].values - corrected_intensity
            elif method == 'IModPoly':
                corrected_intensity = baseObj.IModPoly(poly_degree, iters, conv_thresh)
                baseline = data['Intensity'].values - corrected_intensity
            elif method == 'ZhangFit':
                corrected_intensity = baseObj.ZhangFit(lambda_=lambda_zhang, porder=porder, repitition=repitition)
                baseline = data['Intensity'].values - corrected_intensity
        
        # Apply smoothing
        smoothed_output = savgol_filter(corrected_intensity, window_length, poly_order)
        
        # Create a DataFrame with processed data
        processed_data = pd.DataFrame({
            'Wave': data['Wave'],
            'Original_Intensity': data['Intensity'],
            'Baseline_Corrected': corrected_intensity,
            'Smoothed': smoothed_output
        })
        
        # Create output file path
        file_name = os.path.basename(file_path)
        file_base, file_ext = os.path.splitext(file_name)
        output_file = os.path.join(output_dir, f"{file_base}_bs+sm.xlsx")
        
        # Save to Excel
        processed_data.to_excel(output_file, index=False)
        print(f"Processed data saved to: {output_file}")
        
        # Generate visualization if requested
        if visualize:
            plt.figure(figsize=(12, 12))
            
            # Plot the first set of data - Original, baseline, and corrected
            plt.subplot(2, 1, 1)
            plt.plot(data['Wave'], data['Intensity'], label='Original Data', color='grey')
            plt.plot(data['Wave'], baseline, 'g--', label='Baseline')
            plt.plot(data['Wave'], corrected_intensity, color='blue', label='Original Data - Baseline')
            
            plt.legend(fontsize=12)
            plt.ylabel('Intensity', fontsize=12)
            plt.title(f'Spectrum Analysis using {method} Method', fontsize=14)
            
            # Plot the second set of data - Smoothed data
            plt.subplot(2, 1, 2)
            plt.plot(data['Wave'], smoothed_output, 'b--', label='Smoothed Data')
            
            plt.legend(fontsize=12)
            plt.xlabel('Raman Shift', fontsize=12)
            plt.ylabel('Intensity', fontsize=12)
            
            plt.tight_layout()
            
            # Save the plot
            plot_file = os.path.join(output_dir, f"{file_base}_plot.png")
            plt.savefig(plot_file)
            plt.close()
            print(f"Plot saved to: {plot_file}")
        
        return True
    
    except Exception as e:
        print(f"Error processing {file_path}: {str(e)}")
        return False

# Function to process all files in a folder
def process_folder(input_dir, output_dir, method, poly_degree, iters, conv_thresh, lambda_zhang, porder, repitition,
                   lam_als, p_als, window_length, poly_order):
    
    # Create output directory if it doesn't exist
    if not os.path.exists(output_dir):
        os.makedirs(output_dir)
    
    # Get all files with supported extensions
    file_patterns = [
        os.path.join(input_dir, '*.csv'),
        os.path.join(input_dir, '*.xlsx'),
        os.path.join(input_dir, '*.txt')
    ]
    
    all_files = []
    for pattern in file_patterns:
        all_files.extend(glob.glob(pattern))
    
    if not all_files:
        print(f"No supported files found in {input_dir}")
        return
    
    print(f"Found {len(all_files)} files to process")
    
    # Print parameter information
    half_line = '-' * 40
    stars = '*' * 40
    print(stars)
    print("Processing with these parameters:")
    print(stars)
    print(f"method: {method}")
    print(half_line)
    print("Parameters for ModPoly and IModPoly:")
    print(f"poly_degree: {poly_degree}")
    print(f"iters: {iters}")
    print(f"conv_thresh: {conv_thresh}")
    print(half_line)
    print("Parameters for Zhang Method:")
    print(f"lambda_zhang: {lambda_zhang}")
    print(f"porder: {porder}")
    print(f"repitition: {repitition}")
    print(half_line)
    print("Parameters for ALS Method:")
    print(f"lam_als: {lam_als}")
    print(f"p_als: {p_als}")
    print(half_line)
    print("Parameters for Smoothing:")
    print(f"window_length: {window_length}")
    print(f"poly_order: {poly_order}")
    print(stars)
    
    # Process all files
    success_count = 0
    for i, file_path in enumerate(all_files):
        print(f"Processing file {i+1}/{len(all_files)}: {os.path.basename(file_path)}")
        # Visualize the first file only
        visualize = (i == 0)
        if process_file(file_path, method, poly_degree, iters, conv_thresh, lambda_zhang, porder, repitition,
                     lam_als, p_als, window_length, poly_order, output_dir, visualize):
            success_count += 1
    
    print(f"\nProcessing complete: {success_count}/{len(all_files)} files processed successfully")

# Main function to run the batch processing
if __name__ == "__main__":
    
    # Input and output directories
    input_dir = r"C:\Raw"  # Replace with the input directory
    output_dir = r"C:\BL_Removed"  # Replace with the output directory
    
    # Baseline removal method
    method = 'ZhangFit'  # Can be 'ModPoly', 'IModPoly', 'ZhangFit', or 'ALS'
    
    # Parameters for ModPoly and IModPoly
    poly_degree = 3
    iters = 100
    conv_thresh = 0.001
    
    # Parameters for ZhangFit
    lambda_zhang = 100
    porder = 3
    repitition = 100
    
    # Parameters for ALS
    lam_als = 10000
    p_als = 0.001
    
    # Parameters for Savitzky-Golay filter
    window_length = 21
    poly_order = 3
    
    # Run the processing
    process_folder(input_dir, output_dir, method, poly_degree, iters, conv_thresh, lambda_zhang, porder, repitition,
                  lam_als, p_als, window_length, poly_order)

## 2 - Compare spectra based on top N peaks and Pearson correlation.

In [ ]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from scipy.signal import find_peaks
from scipy.stats import pearsonr
from scipy.interpolate import interp1d
from tqdm import tqdm


def find_top_peaks(df, wave_min, wave_max, num_peaks=10, exclude_ranges=None, name=None):
    subset = df[(df["Wave"] >= wave_min) & (df["Wave"] <= wave_max)].copy()
    if len(subset) < 2:
        return None, None, None, None

    if subset["Smoothed"].max() != subset["Smoothed"].min():
        subset["Normalized"] = (subset["Smoothed"] - subset["Smoothed"].min()) / (subset["Smoothed"].max() - subset["Smoothed"].min())
    else:
        subset["Normalized"] = subset["Smoothed"]

    #prominence threshold (5% of the normalized range) -- if needed you can choose 0.05 or any percentage instead of zero
    prominence_threshold = 0.00 * (subset["Normalized"].max() - subset["Normalized"].min())
    
    peaks, _ = find_peaks(subset["Normalized"], distance=5, prominence=prominence_threshold)
    if len(peaks) == 0:
        return None, None, None, None

    peak_positions = subset["Wave"].iloc[peaks].values
    peak_intensities = subset["Normalized"].iloc[peaks].values

    # Exclusion of ranges
    if exclude_ranges:
        for r_min, r_max in exclude_ranges:
            exclude_mask = (peak_positions < r_min) | (peak_positions > r_max)
            peak_positions = peak_positions[exclude_mask]
            peak_intensities = peak_intensities[exclude_mask]

    if len(peak_positions) == 0:
        return subset, [], np.array([]), np.array([]), 0

    top_indices = np.argsort(peak_intensities)[::-1][:num_peaks]
    top_positions = peak_positions[top_indices]
    top_intensities = peak_intensities[top_indices]

    actual_peaks_used = len(top_positions)

    return subset, None, top_positions, top_intensities, actual_peaks_used


def save_peaks_vertical_grouped(peaks_dict, output_peaks_file, group_size=30, file_prefix=""):
    from openpyxl import Workbook

    peak_names = sorted(peaks_dict.keys())
    wb = Workbook()
    wb.remove(wb.active)

    sheet_count = 0
    for group_start in range(0, len(peak_names), group_size):
        sheet_count += 1
        ws = wb.create_sheet(title=f"Group_{sheet_count}")
        row_idx = 1

        for name in peak_names[group_start:group_start + group_size]:
            result = peaks_dict[name]
            if result is None:
                continue
            _, _, peak_positions, peak_intensities, _ = result  
            peak_df = pd.DataFrame({
                "Peak Position (cm⁻¹)": peak_positions,
                "Peak Intensity (Normalized)": peak_intensities
            }).sort_values(by="Peak Position (cm⁻¹)").reset_index(drop=True)

            ws.cell(row=row_idx, column=1, value=f"{file_prefix}{name}")
            row_idx += 1
            ws.cell(row=row_idx, column=1, value="Peak Position (cm⁻¹)")
            ws.cell(row=row_idx, column=2, value="Peak Intensity (Normalized)")
            row_idx += 1

            for _, row in peak_df.iterrows():
                ws.cell(row=row_idx, column=1, value=row["Peak Position (cm⁻¹)"])
                ws.cell(row=row_idx, column=2, value=row["Peak Intensity (Normalized)"])
                row_idx += 1

            row_idx += 1

    wb.save(output_peaks_file)
    print(f"\nGrouped and sorted peak data saved to {output_peaks_file}")


def save_unknown_peaks_vertical_grouped(unknown_peaks_dict, output_peaks_file, group_size=30):
    save_peaks_vertical_grouped(unknown_peaks_dict, output_peaks_file, group_size, "Polymer: ")


def save_known_peaks_vertical_grouped(known_peaks_dict, output_peaks_file, group_size=30):
    save_peaks_vertical_grouped(known_peaks_dict, output_peaks_file, group_size, "Known Polymer: ")


def compare_raman_spectra(unknown_df, known_result, peak_tolerance=10, num_top_peaks=10,
                          range_min=700, range_max=1700, peak_weight=0.70, pearson_weight=0.30,
                          exclude_ranges=None):
    total_weight = peak_weight + pearson_weight
    if total_weight != 1.0:
        peak_weight /= total_weight
        pearson_weight /= total_weight

    unknown_result = find_top_peaks(unknown_df, range_min, range_max, num_top_peaks, exclude_ranges=exclude_ranges)
    if unknown_result[0] is None or known_result[0] is None:
        return None

    unknown_subset, _, unknown_peaks, _, _ = unknown_result  
    known_subset, _, known_peaks, _, _ = known_result  

    if len(unknown_peaks) > 0 and len(known_peaks) > 0:
        distances = np.abs(unknown_peaks[:, np.newaxis] - known_peaks)
        matched_peaks = np.sum(np.any(distances <= peak_tolerance, axis=1))
        
        denominator = max(len(unknown_peaks), len(known_peaks))        
        # Ensure denominator is not less than numerator
        denominator = max(denominator, matched_peaks)
        
        peak_score = (matched_peaks / denominator) * 100 if denominator > 0 else 0
    else:
        peak_score = 0

    xmin = max(unknown_subset["Wave"].min(), known_subset["Wave"].min())
    xmax = min(unknown_subset["Wave"].max(), known_subset["Wave"].max())
    if xmax <= xmin:
        return 0

    try:
        f1 = interp1d(unknown_subset["Wave"], unknown_subset["Normalized"], kind='linear', bounds_error=True)
        f2 = interp1d(known_subset["Wave"], known_subset["Normalized"], kind='linear', bounds_error=True)
        x_common = np.linspace(xmin, np.nextafter(xmax, xmin), 1000)
        y1 = f1(x_common)
        y2 = f2(x_common)
        corr, _ = pearsonr(y1, y2)
        pearson_score = (corr + 1) * 50
    except:
        pearson_score = 0

    final_score = peak_weight * peak_score + pearson_weight * pearson_score
    return final_score


def plot_best_match(unknown_result, known_result, unknown_name, known_name, similarity_score,
                    peak_tolerance, num_top_peaks, range_min, range_max,
                    peak_weight, pearson_weight, threshold, output_folder):
    unknown_subset, _, unknown_pos, unknown_int, _ = unknown_result  
    known_subset, _, known_pos, known_int, _ = known_result  

    if len(unknown_pos) > 0 and len(known_pos) > 0:
        distances = np.abs(unknown_pos[:, np.newaxis] - known_pos)
        matched_peaks = np.sum(np.any(distances <= peak_tolerance, axis=1))
        
        denominator = len(unknown_pos)    
        # Ensure denominator is not less than numerator
        denominator = max(denominator, matched_peaks)
        
        peak_score = (matched_peaks / denominator) * 100 if denominator > 0 else 0
    else:
        peak_score = 0

    xmin = max(unknown_subset["Wave"].min(), known_subset["Wave"].min())
    xmax = min(unknown_subset["Wave"].max(), known_subset["Wave"].max())

    try:
        f1 = interp1d(unknown_subset["Wave"], unknown_subset["Normalized"], kind='linear', bounds_error=True)
        f2 = interp1d(known_subset["Wave"], known_subset["Normalized"], kind='linear', bounds_error=True)
        x_common = np.linspace(xmin, np.nextafter(xmax, xmin), 1000)
        y1 = f1(x_common)
        y2 = f2(x_common)
        corr, _ = pearsonr(y1, y2)
        pearson_score = (corr + 1) * 50
    except:
        pearson_score = 0
        corr = 0

    plt.figure(figsize=(12, 8))
    plt.subplot(2, 1, 1)
    plt.plot(unknown_subset["Wave"], unknown_subset["Normalized"], label=f"Unknown: {unknown_name}", alpha=0.7)
    plt.plot(known_subset["Wave"], known_subset["Normalized"], label=f"Known: {known_name}", alpha=0.7)
    plt.scatter(unknown_pos, unknown_int, color='blue', marker='x', label="Unknown Peaks")
    plt.scatter(known_pos, known_int, color='red', marker='o', label="Known Peaks")
    #plt.figtext(0.5, 0.45, f"Full Spectrum Correlation: r={corr:.2f}", ha='center', fontsize=10, bbox=dict(facecolor='white', alpha=0.8))
    plt.xlabel("Raman Shift (cm⁻¹)")
    plt.ylabel("Normalized Intensity")
    plt.title(f"{unknown_name} vs {known_name} (Similarity: {similarity_score:.2f}%)")
    plt.legend()
    plt.grid(alpha=0.3)
    plt.xlim(range_min, range_max)
    plt.ylim(0, 1.05)

    plt.subplot(2, 1, 2)
    metrics = ['Peak Match', 'Full Spectrum Corr.', 'Total Similarity']
    scores = [peak_score, pearson_score, similarity_score]
    weights = [peak_weight * 100, pearson_weight * 100, 100]
    plt.bar(metrics, scores, alpha=0.7)
    plt.axhline(y=threshold, color='r', linestyle='--', alpha=0.7, label=f'{threshold}% Threshold')
    for i, v in enumerate(scores):
        plt.text(i, v + 2, f"{v:.2f}%\n(w={weights[i]:.0f}%)", ha='center', va='bottom', fontsize=9)
    plt.ylabel("Score (%)")
    plt.title("Similarity Metrics (Full Spectrum)")
    plt.ylim(0, 105)
    plt.grid(axis='y', alpha=0.3)
    plt.tight_layout()
    if output_folder:
        plt.savefig(os.path.join(output_folder, f"{unknown_name}_vs_{known_name}.png"))
        plt.close()
    else:
        plt.show()


def analyze_folders(unknown_folder, known_folder, output_file,
                    threshold=70, visualize=True, peak_tolerance=10,
                    num_top_peaks=10, range_min=700, range_max=1700,
                    peak_weight=0.70, pearson_weight=0.30,
                    plot_folder=None, exclude_ranges=None):
    results = []
    unknown_files = [f for f in os.listdir(unknown_folder) if f.endswith('.xlsx')]
    known_files = [f for f in os.listdir(known_folder) if f.endswith('.xlsx')]

    if visualize and plot_folder and not os.path.exists(plot_folder):
        os.makedirs(plot_folder)

    all_peak_counts = []

    known_precomputed = {}
    for f in tqdm(known_files, desc="Precomputing knowns"):
        df = pd.read_excel(os.path.join(known_folder, f))
        name = f.replace('_bs+sm.xlsx', '')
        result = find_top_peaks(df, range_min, range_max, num_top_peaks,
                               exclude_ranges=exclude_ranges, name=name)
        known_precomputed[name] = result
        if result[4] is not None:  
            all_peak_counts.append(result[4])

    unknown_peaks_dict = {}
    for f in unknown_files:
        df = pd.read_excel(os.path.join(unknown_folder, f))
        name = f.replace('_bs+sm.xlsx', '')
        result = find_top_peaks(df, range_min, range_max, num_top_peaks,
                               exclude_ranges=exclude_ranges, name=name)
        unknown_peaks_dict[name] = result
        if result[4] is not None:  
            all_peak_counts.append(result[4])

    for u_file in tqdm(unknown_files, desc="Processing unknowns"):
        u_name = u_file.replace('_bs+sm.xlsx', '')
        u_df = pd.read_excel(os.path.join(unknown_folder, u_file))
        for k_name, k_result in known_precomputed.items():
            if u_file == f"{k_name}_bs+sm.xlsx":
                continue
            score = compare_raman_spectra(u_df, k_result, peak_tolerance, num_top_peaks,
                                          range_min, range_max, peak_weight, pearson_weight,
                                          exclude_ranges=exclude_ranges)
            if score is not None:
                results.append([k_name, u_name, score])

    results_df = pd.DataFrame(results, columns=["Known Polymer", "Unknown Polymer", "Similarity (%)"])
    best_matches = {}
    for u_name in set(results_df["Unknown Polymer"]):
        matches = results_df[results_df["Unknown Polymer"] == u_name]
        if not matches.empty:
            best_row = matches.loc[matches["Similarity (%)"].idxmax()]
            k_name = best_row["Known Polymer"]
            score = best_row["Similarity (%)"]
            best_matches[u_name] = (k_name if score >= threshold else None, score)
            if visualize and k_name:
                plot_best_match(unknown_peaks_dict[u_name], known_precomputed[k_name], u_name, k_name, score,
                                peak_tolerance, num_top_peaks, range_min, range_max,
                                peak_weight, pearson_weight, threshold, plot_folder)

    pivot = results_df.pivot(index="Known Polymer", columns="Unknown Polymer", values="Similarity (%)")
    best_row = pd.Series({u: best_matches[u][0] if best_matches[u][0] else "" for u in best_matches}, name="Best Match")
    final_df = pd.concat([pivot, best_row.to_frame().T])

    with pd.ExcelWriter(output_file, engine='xlsxwriter') as writer:
        final_df.to_excel(writer, sheet_name='Results')
        wb = writer.book
        ws = writer.sheets['Results']
        percent_fmt = wb.add_format({'num_format': '0.00'})
        highlight_fmt = wb.add_format({'num_format': '0.00', 'bold': True, 'bg_color': 'yellow'})
        for r in range(1, final_df.shape[0]):
            for c in range(1, final_df.shape[1] + 1):
                val = final_df.iloc[r - 1, c - 1]
                if pd.notna(val) and isinstance(val, (int, float)):
                    fmt = highlight_fmt if val >= threshold else percent_fmt
                    ws.write(r, c, val, fmt)

    print(f"\nResults saved to {output_file}")
    
    # Print peak count summary (actual peaks used in comparison)
    if all_peak_counts:
        from collections import Counter
        peak_counts = Counter(all_peak_counts)
        print(f"\n Peak Detection Summary (peaks used in comparison):")
        print(f"Requested peaks per sample: {num_top_peaks}")
        for peak_count, count in sorted(peak_counts.items()):
            print(f"  {count} samples used {peak_count} peaks")
    
    return unknown_peaks_dict, known_precomputed


if __name__ == "__main__":
    unknown_folder = r'C:\Unkonw' #Unknown polymers folder but make sure use them one after the baseline removal and smoothing from step 1.0
    known_folder = r'C:\Known' #known polymers folder but make sure to use them after the baseline removal and smoothing from step 1.0
    output_file = r'C:\Output.xlsx' #Excel file to save the results
    plot_folder = r'C:\PLot' #Folder to save the Plots
    peak_output_file_grouped = r'C:\output_peaks_unknown_vertical_grouped.xlsx' #Excel file to Recored te unkown polymers peaks
    known_peak_output_file_grouped =  r'C:\output_peaks_known_vertical_grouped.xlsx' #Excel file to Recored te kown polymers peaks

    range_min = 800 # Choose the polymer range you are intrested in
    range_max = 1800 # Choose the polymer range you are intrested in
    num_top_peaks = 12 # Choose the number of peaks for the analysis
    exclude_ranges = [(x1,x2)] # Exclude this range [x1 to x2] from the analysis

    unknown_peaks_dict, known_peaks_dict = analyze_folders(
        unknown_folder,
        known_folder,
        output_file,
        threshold=70, # Change the threshold if needed
        visualize=True,
        peak_tolerance=10,
        num_top_peaks=num_top_peaks,
        range_min=range_min,
        range_max=range_max,
        peak_weight=0.70,  # Change the weight if needed
        pearson_weight=0.30, # Change the weight if needed
        plot_folder=plot_folder,
        exclude_ranges=exclude_ranges
    )

    # Save unknown peaks
    save_unknown_peaks_vertical_grouped(
        unknown_peaks_dict,
        peak_output_file_grouped,
        group_size=30
    )

    # Save known peaks
    save_known_peaks_vertical_grouped(
        known_peaks_dict,
        known_peak_output_file_grouped,
        group_size=30
    )